In [39]:
import datasets

dataset = datasets.load_dataset("deepmind/narrativeqa")

dataset

DatasetDict({
    train: Dataset({
        features: ['document', 'question', 'answers'],
        num_rows: 32747
    })
    test: Dataset({
        features: ['document', 'question', 'answers'],
        num_rows: 10557
    })
    validation: Dataset({
        features: ['document', 'question', 'answers'],
        num_rows: 3461
    })
})

In [40]:
from IPython.display import display

df = dataset["validation"].to_pandas()
df.head()

,document,question,answers
0,{'id': '00fb61fa7bee266ad995e52190ebb73606b60b...,{'text': 'WHO NORMALLY DELIVERS THE OPENING PR...,"[{'text': 'THE ACTOR WEARING THE BLACK CLOAK',..."
1,{'id': '00fb61fa7bee266ad995e52190ebb73606b60b...,{'text': 'WHAT NAME WAS CYNTHIA MORE FAMOUSLY ...,"[{'text': 'THE GODDESS DIANA', 'tokens': ['THE..."
2,{'id': '00fb61fa7bee266ad995e52190ebb73606b60b...,"{'text': 'WHO DOES ECHO WEEP FOR?', 'tokens': ...","[{'text': 'NARCISSUS', 'tokens': ['NARCISSUS']..."
3,{'id': '00fb61fa7bee266ad995e52190ebb73606b60b...,{'text': 'WHAT DOES A DRINK FROM NARCISSUS'S S...,"[{'text': 'FALL IN LOVE WITH THEMSELVES', 'tok..."
4,{'id': '00fb61fa7bee266ad995e52190ebb73606b60b...,{'text': 'IN WHAT VALLEY DID THE SOLEMN REVELS...,"[{'text': 'GARGAPHIE IN GREECE', 'tokens': ['G..."


In [41]:
df['context'] = df['document'].apply(lambda x: x['text'])
df['question'] = df['question'].apply(lambda x: x['text'])
df.head()

,document,question,answers,context
0,{'id': '00fb61fa7bee266ad995e52190ebb73606b60b...,WHO NORMALLY DELIVERS THE OPENING PROLOGUE IN ...,"[{'text': 'THE ACTOR WEARING THE BLACK CLOAK',...",ï»¿The Project Gutenberg EBook of Cynthia's Re...
1,{'id': '00fb61fa7bee266ad995e52190ebb73606b60b...,WHAT NAME WAS CYNTHIA MORE FAMOUSLY KNOWN BY?,"[{'text': 'THE GODDESS DIANA', 'tokens': ['THE...",ï»¿The Project Gutenberg EBook of Cynthia's Re...
2,{'id': '00fb61fa7bee266ad995e52190ebb73606b60b...,WHO DOES ECHO WEEP FOR?,"[{'text': 'NARCISSUS', 'tokens': ['NARCISSUS']...",ï»¿The Project Gutenberg EBook of Cynthia's Re...
3,{'id': '00fb61fa7bee266ad995e52190ebb73606b60b...,WHAT DOES A DRINK FROM NARCISSUS'S SPRING CAUS...,"[{'text': 'FALL IN LOVE WITH THEMSELVES', 'tok...",ï»¿The Project Gutenberg EBook of Cynthia's Re...
4,{'id': '00fb61fa7bee266ad995e52190ebb73606b60b...,IN WHAT VALLEY DID THE SOLEMN REVELS OF CYNTHI...,"[{'text': 'GARGAPHIE IN GREECE', 'tokens': ['G...",ï»¿The Project Gutenberg EBook of Cynthia's Re...


In [42]:
df = df.rename(columns={'question': 'questions'})
df = df.groupby('context').agg({'questions': list}).reset_index()
df.head()

,context,questions
0,"<html>\n\n<head>\n<title>""Domino,"" by Richard ...",[Who planned the robbery that was being invest...
1,<html>\n<head><title>Airplane Script at IMSDb....,"[What is Ted Striker afraid of?, Why is Ted af..."
2,<html>\n<head><title>All About Steve Script at...,"[What is Mary Horowitz's job?, Who is Mary's b..."
3,<html>\n<head><title>American Psycho Script at...,"[Who is the first man Bateman muders?, Who is ..."
4,"<html>\n<head><title>American, The Script at I...","[What does Jack do for a living?, What is the ..."


In [43]:
# Check for duplicates in context column
duplicate_contexts = df['context'].duplicated().sum()
print(f"Number of duplicate contexts: {duplicate_contexts}")
df.shape

Number of duplicate contexts: 0


(115, 2)

In [44]:
df = df.sample(n=50, random_state=42)
df.head()

,context,questions
81,ï»¿The Project Gutenberg EBook of His Masterpi...,[What is the name of the painting that sparks ...
4,"<html>\n<head><title>American, The Script at I...","[What does Jack do for a living?, What is the ..."
40,<html>\n<head><title>Music of the Heart Script...,"[What instrument does Roberta Guaspari play?, ..."
69,ï»¿Project Gutenberg's Tom Swift and his Photo...,[Who does Tom want to have access to his paten...
10,<html>\n<head><title>Confidence Script at IMSD...,"[Why was one of the four grifters shot?, How d..."


In [45]:
import pandas as pd

# Save DataFrame to parquet file
df.to_parquet('data/narrativeqa.parquet')
df = pd.read_parquet('data/narrativeqa.parquet')
df.head()

,context,questions
81,ï»¿The Project Gutenberg EBook of His Masterpi...,[What is the name of the painting that sparks ...
4,"<html>\n<head><title>American, The Script at I...","[What does Jack do for a living?, What is the ..."
40,<html>\n<head><title>Music of the Heart Script...,"[What instrument does Roberta Guaspari play?, ..."
69,ï»¿Project Gutenberg's Tom Swift and his Photo...,[Who does Tom want to have access to his paten...
10,<html>\n<head><title>Confidence Script at IMSD...,"[Why was one of the four grifters shot?, How d..."


In [46]:
df['context_length'] = df['context'].apply(lambda x: len(x))
df.describe()

,context_length
count,5.000000e+01
mean,3.637226e+05
std,3.511947e+05
min,7.084000e+04
25%,1.906202e+05
50%,2.369300e+05
75%,3.624858e+05
max,1.960965e+06


In [47]:
from counter import get_and_increment_counter
from llama_stack_client import LlamaStackClient
from llama_stack_client.types.memory_insert_params import Document

client = LlamaStackClient(
    base_url="http://localhost:5001",
)

providers = client.providers.list()
memory_banks_response = client.memory_banks.list()

bank_id = f"bank_{get_and_increment_counter()}"
provider = providers["memory"][0]
client.memory_banks.register(
    memory_bank_id=bank_id,
    params={
        "embedding_model": "all-MiniLM-L6-v2",
        # Is the default for agent config: https://github.com/meta-llama/llama-stack/blob/66d8f4ffd126bff668434b314892a99fe854a034/llama_stack/providers/inline/agents/meta_reference/agent_instance.py#L668
        "chunk_size_in_tokens": 512,
    },
    provider_id=provider.provider_id,
)
bank_id

'bank_35'

In [48]:
documents = [
    Document(
        document_id=str(idx),
        content=context,
        mime_type="text/plain",
        metadata={},
    )
    for idx, context in zip(df.index, df["context"])
]
documents[:5]

[{'document_id': '81',
  'content': 'ï»¿The Project Gutenberg EBook of His Masterpiece, by Emile Zola\n\nThis eBook is for the use of anyone anywhere at no cost and with\nalmost no restrictions whatsoever.  You may copy it, give it away or\nre-use it under the terms of the Project Gutenberg License included\nwith this eBook or online at www.gutenberg.org\n\n\nTitle: His Masterpiece\n\nAuthor: Emile Zola\n\nEditor: Ernest Alfred Vizetelly\n\nTranslator: Ernest Alfred Vizetelly\n\nRelease Date: May 25, 2005 [EBook #15900]\nPosting Date: May 30, 2009\nLast Updated: September 5, 2016\n\nLanguage: English\n\nCharacter set encoding: UTF-8\n\n*** START OF THIS PROJECT GUTENBERG EBOOK HIS MASTERPIECE ***\n\n\n\n\nProduced by Dagny; and David Widger\n\n\n\n\n\n\n\n\nHIS MASTERPIECE\n\nBy Emile Zola\n\n\nEdited, With a Preface, By Ernest Alfred Vizetelly\n\n\n\n\nPREFACE\n\nâ\x80\x98HIS MASTERPIECE,â\x80\x99 which in the original French bears the title of\n_Lâ\x80\x99Oeuvre_, is a strikingly acc

In [49]:
from tqdm import tqdm

for i in tqdm(range(len(documents))):
    client.memory.insert(
        bank_id=bank_id,
        documents=[documents[i]],
    )

  0%|                                                                                         | 0/50 [00:00<?, ?it/s]

 14%|███████████▎                                                                     | 7/50 [00:31<03:54,  5.44s/it]